In [3]:
import sys
sys.path.append('/home/cara/Documents/reddit_analysis_code')
import numpy as np
import pandas as pd
from reddit_dataclass import RedditData as reddit
import pickle
import matplotlib.pyplot as plt

In [4]:
sentiment_datasets = pickle.load(open('sentiment_datasets.p', 'rb'))

In [5]:
thread_cols = ['thread_id', 'id', 'timestamp','author','domain','parent',
       'score', 'subject_sentiment_score', 'body_sentiment_score', 'date']

In [18]:
"""
for key in sentiment_datasets:
    sentiment_datasets[key].data['parent'] = sentiment_datasets[key].data['parent'].str[3:]
pickle.dump(sentiment_datasets, open('sentiment_datasets.p', 'wb'))
"""

In [6]:
thread_datasets = {}

for key in sentiment_datasets:
    print(f'### WORKING ON {key} ###')

    sentiment_data = sentiment_datasets[key].data
    thread_data = sentiment_data[thread_cols]
    thread_data['level'] = -1

    parent_post_id = sentiment_data[sentiment_data.thread_id == sentiment_data.id].id

    thread_data.loc[thread_data.id.isin(parent_post_id), 'level'] = 0

    children = {}

    children[0] = sentiment_data[sentiment_data.parent == sentiment_data.thread_id]
    children[0]['parent_comment'] = children[0].id
    children[0]['level'] = 1

    i = 0
    while len(children[i]) > 0:

        print(f"level {i}  number of children: {len(children[i])}")
        i += 1
        children[i] = sentiment_data[sentiment_data.parent.isin(children[i-1].id)]
        children[i]['level'] = i + 1
        parent_mapper = dict(zip(
            children[i-1].id, children[i-1].parent_comment
        ))
        children[i]["parent_comment"] = children[i].parent.map(parent_mapper)
    
    thread_data['parent_comment'] = np.nan

    for lvl in children:
        thread_data.loc[
            thread_data.id.isin(children[lvl].id), 
            'level'] = lvl + 1
        if lvl == 0:
            thread_data.loc[
                thread_data.id.isin(children[lvl].id),
                'parent_comment'] = thread_data[
                    thread_data.id.isin(children[lvl].id)].id
        else:
            parent_mapper = dict(zip(
                children[lvl].id, children[lvl].parent_comment
            ))
            thread_data.loc[
                thread_data.id.isin(children[lvl].id),
                'parent_comment'] = thread_data[
                    thread_data.id.isin(children[lvl].id)].id.map(
                        parent_mapper)
    
    thread_datasets[key] = thread_data
    


### WORKING ON books ###
level 0  number of children: 45949
level 1  number of children: 19426
level 2  number of children: 11914
level 3  number of children: 7499
level 4  number of children: 4920
level 5  number of children: 3033
level 6  number of children: 1725
level 7  number of children: 1087


/tmp/ipykernel_11823/4055434657.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thread_data['level'] = -1
/home/cara/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/tmp/ipykernel_11823/4055434657.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

level 8  number of children: 642
level 9  number of children: 414
level 10  number of children: 255
level 11  number of children: 159
level 12  number of children: 103
level 13  number of children: 70
level 14  number of children: 50
level 15  number of children: 41
level 16  number of children: 31
level 17  number of children: 25
level 18  number of children: 19
level 19  number of children: 11
level 20  number of children: 7
level 21  number of children: 6
level 22  number of children: 5
level 23  number of children: 3
level 24  number of children: 2
level 25  number of children: 2
level 26  number of children: 2
level 27  number of children: 1
level 28  number of children: 1
level 29  number of children: 1
level 30  number of children: 1
level 31  number of children: 1
level 32  number of children: 1
level 33  number of children: 1
level 34  number of children: 1
level 35  number of children: 1
level 36  number of children: 1
level 37  number of children: 1
level 38  number of child

/tmp/ipykernel_11823/4055434657.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thread_data['parent_comment'] = np.nan
/home/cara/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


### WORKING ON conspiracy ###
level 0  number of children: 131877
level 1  number of children: 74590
level 2  number of children: 53709
level 3  number of children: 37979
level 4  number of children: 27380
level 5  number of children: 19631
level 6  number of children: 13927
level 7  number of children: 10036
level 8  number of children: 7331
level 9  number of children: 5318
level 10  number of children: 4014
level 11  number of children: 2948
level 12  number of children: 2207
level 13  number of children: 1699
level 14  number of children: 1330
level 15  number of children: 1028
level 16  number of children: 821
level 17  number of children: 649
level 18  number of children: 546
level 19  number of children: 443
level 20  number of children: 353
level 21  number of children: 305
level 22  number of children: 254
level 23  number of children: 219
level 24  number of children: 187
level 25  number of children: 160
level 26  number of children: 139
level 27  number of children: 120
lev

In [9]:
pickle.dump(thread_datasets, open('thread_data.p', 'wb'))

In [10]:
unclassified = {}
for key in thread_datasets:
    unclassified[key] = thread_datasets[key][thread_datasets[key].level == -1]

In [13]:
print('subreddit  sentiment  thread_entries     unclassified')
for key in thread_datasets:
    print(f"{key} {len(sentiment_datasets[key].data)}  {len(thread_datasets[key])} {len(unclassified[key])}")

subreddit  sentiment  thread_entries     unclassified
books 105291  105291 3090
conspiracy 423958  423958 12025
crypto 444397  444397 7862
thedonald 1543  1543 17


In [27]:
# parent is in dataset (so chain broken at parent level)
unclassified['books'][unclassified['books'].parent.isin(thread_datasets['books'].id)]

,thread_id,id,timestamp,author,domain,parent,score,subject_sentiment_score,body_sentiment_score,date,level,parent_comment
5189,xsfue6,iqkfs4g,2022-10-01 00:17:38,d630292b3ef8845553f46794f2dd2e9d569d63d0ae979aa7,NaN,iqkeq92,1,NaN,0.000000,2022-10-01,-1,NaN
5191,xsfue6,iqkfved,2022-10-01 00:18:24,d630292b3ef8845553f46794f2dd2e9d569d63d0ae979aa7,NaN,iqkeuyi,1,NaN,0.148000,2022-10-01,-1,NaN
5204,xrzmx2,iqkgij8,2022-10-01 00:23:35,fb867649f8be84d4e92f2655b23b7a28d56b4f2441020101,NaN,iqkevnf,1,NaN,-0.035579,2022-10-01,-1,NaN
5256,xsa2rt,iqkjbxm,2022-10-01 00:47:00,e7cb06f2e1a21e857dd8a820233c261d5a56df52f4384f8f,NaN,iqkiqg6,1,NaN,0.000000,2022-10-01,-1,NaN
5257,xrzmx2,iqkjesn,2022-10-01 00:47:41,5e647aa630004c88dde9edeecddfa520ef805133f7b344ee,NaN,iqkhrc9,1,NaN,0.088250,2022-10-01,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
110255,wdm12n,iuf3ypc,2022-10-30 21:10:48,cfecd0903a56e710a5e6a0289c89de6f6d1384ab1be5c23f,NaN,iucwxgx,1,NaN,0.616600,2022-10-30,-1,NaN
110317,yhks0d,iuf5us1,2022-10-30 21:23:27,db2a038186460d3fd5ec71682826beacdc08af82d3484508,NaN,iuf1ful,1,NaN,0.735100,2022-10-30,-1,NaN
110686,yhks0d,iufg292,2022-10-30 22:34:23,e3045e1efbd1af40da638048d92639d51dfc4d3779b56b13,NaN,iuf5us1,1,NaN,-0.061575,2022-10-30,-1,NaN
110778,yhks0d,iufix1j,2022-10-30 22:54:36,3da63530acb02b423a171a563a7deff4d689c145869cc4be,NaN,iufb6fk,1,NaN,0.371500,2022-10-30,-1,NaN


In [29]:
# parent not in dataset
unclassified['books'][~unclassified['books'].parent.isin(thread_datasets['books'].id)]

,thread_id,id,timestamp,author,domain,parent,score,subject_sentiment_score,body_sentiment_score,date,level,parent_comment
5160,xsa2rt,iqkdr9s,2022-10-01 00:00:43,3d0a150b3f3cd4c5f8147fa3a1056b2590da8e9636c9ca3b,NaN,iqk18l7,1,NaN,-0.708900,2022-10-01,-1,NaN
5165,xsgk9w,iqkdwcf,2022-10-01 00:01:52,5461a84e952f727be9f548b068c04625a708dc0f7df7494b,NaN,iqkdtk0,1,NaN,0.623900,2022-10-01,-1,NaN
5166,xrzmx2,iqkdxhu,2022-10-01 00:02:08,320d45835f9c998f21793b3d2850ead9deee9304590556e3,NaN,iqkaxk3,1,NaN,-0.078400,2022-10-01,-1,NaN
5167,xrmbj1,iqkdzm8,2022-10-01 00:02:38,ffbb9d36eaa76e314dd7d3f8d63f9bb2e6ca61a32ba9afde,NaN,iqgigm6,1,NaN,-0.200950,2022-10-01,-1,NaN
5170,xsa2rt,iqke8me,2022-10-01 00:04:43,1978e64dd53d37b1bce0662177d3270adede4e50085d11d2,NaN,iqk18l7,1,NaN,-0.109400,2022-10-01,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
110616,yhks0d,iufdpvi,2022-10-30 22:17:39,3d9e75f12e3b1e6fb6010481e1ea5250177f81e105838b94,NaN,iuemwr0,1,NaN,0.088750,2022-10-30,-1,NaN
110844,9gb5q7,iufl879,2022-10-30 23:11:00,1fe98be8dbba7a9b4e6e2aa81acfb16ae0832f92461854c4,NaN,e63sv7e,1,NaN,-0.506640,2022-10-30,-1,NaN
110869,9gb5q7,iufm5zw,2022-10-30 23:17:42,1fe98be8dbba7a9b4e6e2aa81acfb16ae0832f92461854c4,NaN,e63sv7e,1,NaN,-0.599600,2022-10-30,-1,NaN
110967,rt4p4u,iufoxo5,2022-10-30 23:38:16,abc96d92f2996b428f3914679057aaad93726aa1fa97e60d,NaN,hqqnjhg,1,NaN,0.766050,2022-10-30,-1,NaN


In [33]:
# check if spread over a range of dates
for key in unclassified:
    date_str = ""
    for date in unclassified[key].date.unique():
        date_str += f"{date}    "
    print(f"## {key} ##")
    print(date_str)

## books ##
2022-10-01    2022-10-02    2022-10-03    2022-10-04    2022-10-05    2022-10-06    2022-10-07    2022-10-08    2022-10-09    2022-10-10    2022-10-11    2022-10-12    2022-10-13    2022-10-14    2022-10-15    2022-10-16    2022-10-17    2022-10-18    2022-10-19    2022-10-20    2022-10-21    2022-10-22    2022-10-23    2022-10-24    2022-10-25    2022-10-26    2022-10-27    2022-10-28    2022-10-29    2022-10-30    
## conspiracy ##
2022-10-01    2022-10-02    2022-10-03    2022-10-04    2022-10-05    2022-10-06    2022-10-07    2022-10-08    2022-10-09    2022-10-10    2022-10-11    2022-10-12    2022-10-13    2022-10-14    2022-10-15    2022-10-16    2022-10-17    2022-10-18    2022-10-19    2022-10-20    2022-10-21    2022-10-22    2022-10-23    2022-10-24    2022-10-25    2022-10-26    2022-10-27    2022-10-28    2022-10-29    2022-10-30    
## crypto ##
2022-10-01    2022-10-02    2022-10-03    2022-10-04    2022-10-05    2022-10-06    2022-10-07    2022-10-08    2022

In [39]:
removed_parent_activity = {}
for key in sentiment_datasets:
    removed_parent_activity[key] = unclassified[key][unclassified[key].parent.isin(sentiment_datasets[key].removed.id)]
    print(f"{key}   {len(removed_parent_activity[key])}")

books   362
conspiracy   508
crypto   773
thedonald   1


In [43]:
# save thread data and unclassified data
clean_thread_data = {}
for key in thread_datasets:
    clean_thread_data[key] = thread_datasets[key][thread_datasets[key].level != -1]

In [47]:
pickle.dump(clean_thread_data, open('clean_thread_data.p', 'wb'))
pickle.dump(unclassified, open('unclassified_thread_data.p', 'wb'))